In [ ]:
from pyquery import PyQuery as pq
from pandas as pd
import requests

import re
from time import sleep
import csv

import datetime
import re
import lxml
from urllib.request import urlopen
from urllib.error import HTTPError

In [ ]:
try:
    insp = pd.read_csv("ftp://dbprftp.state.fl.us/pub/llweb/5fdinspi.csv", 
                               usecols=[2,14,18,80,81])
except IOError:
    print("The file is not accessible.")
insp.columns = ["CountyName", "InspectDate", 
                "NumHighVio", "LicenseID", "VisitID"]

insp[(insp.CountyName == 'Alachua') & (insp.NumHighVio > 0)]

# change date string to date object, sort most recent# chang 
insp['InspectDate'] = pd.to_datetime(insp['InspectDate'])
insp = insp.sort_values('InspectDate', ascending=False)

# prefer to let user set timedelta or just remove. 
today = pd.to_datetime('today')
startDay = datetime.date.today() - datetime.timedelta(days=14)
insp = insp[(insp['InspectDate'] > startDay) & (insp['InspectDate'] < today)]
len(insp)

# takes LicenseID and VisitID, passes it into the urls for detailed reports#
result = []
for index, rows in insp.iterrows():
    visitID = rows['VisitID']
    licID = rows['LicenseID']
    urls = "https://www.myfloridalicense.com/inspectionDetail.asp? \
        InspVisitID= %s &licid= %s" % (visitID, licID)
    urls = urls.replace(' ', '')
    result.append(urls)

urlList = result

In [ ]:
for inspection in inspections:
    basehtml = requests.get(urlList).content
    